# TFLite Optimizations and Quantization

> In this post, we will explore optimization impact on size, performance and accuracy. This is the summary of lecture "Applications of TinyML" from edX.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, edX, Deep_Learning, Tensorflow, tinyML]
- image: 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Tensorflow : v" + tf.__version__)

Tensorflow : v2.3.1


## Data Load & Preprocessing

In [2]:
# format images to have normalized pixels
def format_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return  image, label

# load in our dataset
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

# display how much data we have
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print(num_examples)
print(num_classes)

# split the data in training, validation, and test datasets
BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

# display the shape of our data
for image_batch, label_batch in train_batches.take(1):
    pass
image_batch.shape

23262
2


TensorShape([32, 224, 224, 3])

## Define pre-trained model

In [3]:
module_selection = ("mobilenet_v2", 224, 1280) 
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=False)

print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280
Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


## Train & Save model

Since we are doing transfer learning to fine tune a pre-trained model to our dataset we only need to use 5 Epochs.

In [4]:
EPOCHS = 5

hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)

Epoch 1/5
582/582 [==============================] - 18s 31ms/step - loss: 0.0520 - accuracy: 0.9822 - val_loss: 0.0369 - val_accuracy: 0.9858
Epoch 2/5
582/582 [==============================] - 11s 19ms/step - loss: 0.0307 - accuracy: 0.9901 - val_loss: 0.0394 - val_accuracy: 0.9862
Epoch 3/5
582/582 [==============================] - 12s 20ms/step - loss: 0.0249 - accuracy: 0.9913 - val_loss: 0.0372 - val_accuracy: 0.9884
Epoch 4/5
582/582 [==============================] - 11s 19ms/step - loss: 0.0209 - accuracy: 0.9927 - val_loss: 0.0377 - val_accuracy: 0.9880
Epoch 5/5
582/582 [==============================] - 11s 20ms/step - loss: 0.0188 - accuracy: 0.9936 - val_loss: 0.0407 - val_accuracy: 0.9871


In [5]:
CATS_VS_DOGS_SAVED_MODEL = "exp_saved_model"
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: exp_saved_model\assets


INFO:tensorflow:Assets written to: exp_saved_model\assets


## Model optimization & Quantization

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)

### No Optimization & Quantization

In [11]:
tflite_model = converter.convert()
tflite_models_dir = pathlib.Path("./")

tflite_model_file = tflite_models_dir/'model_noop.tflite'
tflite_model_file.write_bytes(tflite_model)

8944916

### Default Optimization

In [12]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
tflite_models_dir = pathlib.Path("./")

tflite_model_file = tflite_models_dir/'model_op_default.tflite'
tflite_model_file.write_bytes(tflite_model)

2389808

### Default Optimization and quantization

In [13]:
def representative_data_gen():
    for input_value, _ in test_batches.take(100):
        yield [input_value]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_model = converter.convert()
tflite_models_dir = pathlib.Path("./")

tflite_model_file = tflite_models_dir/'model_op_default_quant.tflite'
tflite_model_file.write_bytes(tflite_model)

2923024

## Evaluate model

In [14]:
class_names = ['cat', 'dog']

def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    img = np.squeeze(img)

    plt.imshow(img, cmap=plt.cm.binary)
    
    predicted_label = np.argmax(predictions_array)
    
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                         100*np.max(predictions_array),
                                         class_names[true_label]), color=color)

In [15]:
for tflite_model_file in ['model_noop.tflite', 'model_op_default.tflite', 'model_op_default_quant.tflite']:
    interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
    interpreter.allocate_tensors()

    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    predictions = []

    test_labels, test_imgs = [], []
    for img, label in tqdm(test_batches.take(100)):
        interpreter.set_tensor(input_index, img)
        interpreter.invoke()
        predictions.append(interpreter.get_tensor(output_index))

        test_labels.append(label.numpy()[0])
        test_imgs.append(img)
        
    score = 0
    for item in range(0,99):
        prediction=np.argmax(predictions[item])
        label = test_labels[item]
        if prediction==label:
            score +=1

    print("Out of 100 predictions I got " + str(score) + " correct on " + tflite_model_file)

  1%|▊                                                                                 | 1/100 [00:00<00:18,  5.40it/s]

Out of 100 predictions I got 99 correct on model_noop.tflite


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Out of 100 predictions I got 93 correct on model_op_default.tflite


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:12<00:00,  1.32s/it]

Out of 100 predictions I got 99 correct on model_op_default_quant.tflite
